In [ ]:
import torch
import lib.utils.bookkeeping as bookkeeping
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lib.models.models as models
import lib.models.model_utils as model_utils
from lib.datasets import mnist, maze, protein, synthetic
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils
import os
from lib.datasets.maze import maze_acc
from ruamel.yaml.scalarfloat import ScalarFloat

In [ ]:
# creating path

path = 'SavedModels/BIN-MNIST/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-17' # 2
config_name = 'config_001_unet3M.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_199999_unet3M.pt' 

path = 'SavedModels/MAZEprotein/'
date = "2023-12-23"
config_name = "config_001_score.yaml"
model_name = "model_199999_score.pt"

path = 'SavedModels/MAZEunet/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-23' # 2
config_name = 'config_001.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_9999.pt' #  model_299999_hollowMLEProb.pt



path = 'SavedModels/MNISTHollow/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2024-01-03' # 2
config_name = 'config_001_hollow.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_539999_hollow.pt'

path = 'SavedModels/MNIST/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-25' # 2
config_name = 'config_001_unet14Mlogits.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_599999_unet14Mlogits.pt'


path = 'SavedModels/MAZEunet/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-30' # 2
config_name = 'config_001_lastunet.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_399999_lastunet.pt'

path = 'SavedModels/MNISTHollow/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2024-01-03' # 2
config_name = 'config_001_hollow.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_539999_hollow.pt'

path = 'SavedModels/MAZE/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-22' # 2
config_name = 'config_001_hollow8M.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_299999_hollow8M.pt' 

path = "SavedModels/Synthetic/"
date = '2023-12-20' # 2
config_name = 'config_001_hollowCEProb500K.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_199999_hollowCEProb500K.pt'


path = "SavedModels/SyntheticMasked/"
date = '2023-12-17' # 2
config_name = 'config_001_masked.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_199999_masked.pt'

path = "SavedModels/SyntheticBert/"
date = '2023-12-28' # 2
config_name = 'config_001_bert500K.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_199999_bert500K.pt'

path = "SavedModels/SyntheticRMDirect/"
date = '2023-12-20' # 2023-10-30 'Hollow-2023-10-29'
config_name = 'config_001_hollowCEDirect500K.yaml' # 'config_001_maze.yaml' 'config_001_rate001.yaml'
model_name = 'model_199999_hollowCEDirect500K.pt' 

path = "SavedModels/SyntheticRMDirect/"
date = '2023-12-20' # 2023-10-30 'Hollow-2023-10-29'
config_name = 'config_001_hollowCEDirect500K.yaml' # 'config_001_maze.yaml' 'config_001_rate001.yaml'
model_name = 'model_199999_hollowCEDirect500K.pt' 

path = "SavedModels/Synthetic/"
date = '2023-12-20' # 2
config_name = 'config_001_hollowCEProb500K.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_199999_hollowCEProb500K.pt'

path = 'SavedModels/MNIST/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-25' # 2
config_name = 'config_001_unet14Mlogits.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_599999_unet14Mlogits.pt'

config_path = os.path.join(path, date, config_name)
checkpoint_path = os.path.join(path, date, model_name)



In [ ]:
# creating models
cfg = bookkeeping.load_config(config_path)
cfg.sampler.name = 'LBJF' #ExactSampling' # ElboLBJF CRMTauL CRMLBJF
cfg.sampler.num_corrector_steps = 0
cfg.sampler.corrector_entry_time = ScalarFloat(0.0)
cfg.sampler.num_steps = 500
cfg.sampler.is_ordinal = True

#print(cfg)
device = torch.device(cfg.device)
print(device)

model = model_utils.create_model(cfg, device)
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

#modified_model_state = utils.remove_module_from_keys(loaded_state['model'])
#model.load_state_dict(modified_model_state)
#optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)
optimizer = torch.optim.Adam(model.parameters(), cfg.optimizer.lr)

sampler = sampling_utils.get_sampler(cfg)

state = {"model": model, "optimizer": optimizer, "n_iter": 0}
state = bookkeeping.load_state(state, checkpoint_path)
state['model'].eval()

# Maze:
# 80% TauL
# 84% LBJF
# 96%

In [4]:
steps = [2000]
n_samples = 2000 # 1mal Taul 750 2500 samples und einmal LBJF 1000 25000 samples 
for i in range(5):
    for _, n_steps in enumerate(steps):
        for _, n_name in enumerate(['LBJF', 'TauL']):
            cfg.sampler.name = n_name 
            cfg.sampler.num_steps = n_steps
            sampler = sampling_utils.get_sampler(cfg)
            samples, changes= sampler.sample(model, n_samples)
            saved_samples = samples
            np.save(f'unet_{cfg.sampler.name}{cfg.sampler.num_steps}_mnist{i+1}', saved_samples.reshape(n_samples, 1, cfg.data.image_size, cfg.data.image_size))
#print(samples)

1978it [28:46,  1.14it/s]

In [ ]:
#np.save(f'lib/is_mnist/data/{cfg.model.name}_{cfg.sampler.name}{cfg.sampler.num_steps}_corrector10.npy', saved_samples)
#np.save(f'{cfg.model.name}_{cfg.sampler.name}{cfg.sampler.num_steps}_mnist3', saved_samples.reshape(n_samples, 1, cfg.data.image_size, cfg.data.image_size))

#steps = [10, 20, 30, 50, 100, 250, 500]#, 750, 1000]
steps = [1000]
sampler_name = ['LBJF']#, 'TauL']#['LBJF', 'TauL'] # 'LBJF', 
for sampler_n_i in sampler_name:
    for step in steps:
        print(f'unet_{sampler_n_i}{step}_mnist.npy')
        data1 = np.load(f'unet_{sampler_n_i}{step}_mnist1.npy')
        data2 = np.load(f'unet_{sampler_n_i}{step}_mnist2.npy')
        data3 = np.load(f'unet_{sampler_n_i}{step}_mnist3.npy')
        data4 = np.load(f'unet_{sampler_n_i}{step}_mnist4.npy')
        data5 = np.load(f'unet_{sampler_n_i}{step}_mnist5.npy')
        data = np.vstack((data1, data2, data3, data4, data5))
        print(data.shape)
        np.save(f'mnist_unet_{sampler_n_i}{step}.npy', data)


In [ ]:
data1 = np.load(f'unet_LBJF1000_mnist.npy') # 750 Taul 7500 1000: TauL 6000
print(data1.shape)

In [ ]:
is_img = cfg.data.is_img
n_samples = 100
if is_img:
    samples = samples.reshape(-1, 1, cfg.data.image_size, cfg.data.image_size)
    saving_train_path = os.path.join(cfg.saving.sample_plot_path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
    fig = plt.figure(figsize=(9, 9)) 
    for i in range(n_samples):
        plt.subplot(int(np.sqrt(n_samples)), int(np.sqrt(n_samples)), 1 + i)
        plt.axis("off")
        plt.imshow(np.transpose(samples[i, ...], (1,2,0)), cmap="gray")
 
 
    plt.savefig(saving_train_path)
    plt.show()
    plt.close()
else:
    bm, inv_bm = synthetic.get_binmap(cfg.model.concat_dim, cfg.data.binmode)
    print(inv_bm)
    samples = synthetic.bin2float(samples.astype(np.int32), inv_bm, cfg.model.concat_dim, cfg.data.int_scale)

 
    saving_plot_path = os.path.join(path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
    saving_np_path = os.path.join(path, f"samples_{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.npy")
    np.save(f'{saving_np_path}', samples)

    #aving_plot_path = '/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/TAUnSDDM/SavedModels/MNIST/'
    synthetic.plot_samples(samples, saving_plot_path, im_size=cfg.data.plot_size, im_fmt="png")

In [ ]:
"""
#np.save('PNGs/changing_dim_taul_maze50_ordinal.npy', changes)
#np.save('PNGs/changing_dim_lbjf_maze50.npy', changes)
changes_jump_taul = np.load('PNGs/changing_dim_taul_maze50_ordinal.npy')
changes_jump_lbjf = np.load('PNGs/changing_dim_lbjf_maze50.npy')
steps = np.linspace(1, 0, len(changes_jump_lbjf[:-1]))
plt.gca().invert_xaxis()
plt.plot(steps, changes_jump_lbjf[:-1], label='Euler')
plt.plot(steps, changes_jump_taul[:-1], label='Tau-Leaping', color='green')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Avg. Proportion of Changing Dimensions', fontsize=13)
plt.legend()
plt.grid(True)
plt.savefig('PNGs/changing_dim_maze50_ordinal.png')
plt.show()
"""


In [ ]:
#np.save('PNGs/state_changes_taul.npy', changes.detach().cpu().numpy())
"""
s_changes_taul = np.load('PNGs/state_changes_taul.npy')
changes[-1, 0, :].reshape(28,28)
shaped_changes = s_changes_taul[:, 0, :].reshape(-1, 28,28)
indices = np.where((shaped_changes[-1, :, :] > 20) & (shaped_changes[-1, :, :] < 250))
print(indices[0])
print(indices[1])
steps = np.linspace(1, 0, cfg.sampler.num_steps)
plt.gca().invert_xaxis()
#for i in range(max_number):
plt.plot(steps, shaped_changes[:, 14, 14])
plt.plot(steps, shaped_changes[:, 9, 8])
plt.plot(steps, shaped_changes[:, 10, 10])
plt.plot(steps, shaped_changes[:, 13, 7])
plt.plot(steps, shaped_changes[:, 24, 20])
plt.xlabel('Time', fontsize=13)
plt.ylabel('State',fontsize=13)
plt.legend()
plt.grid(True)
plt.savefig('PNGs/state_changes_taul.png')
plt.show()
"""

In [ ]:
"""
#np.save('PNGs/state_changes_lbjf.npy', changes)
s_changes_lbjf = np.load('PNGs/state_changes_lbjf.npy')
changes[-1, 0, :].reshape(28,28)
shaped_changes = s_changes_lbjf[:, 0, :].reshape(-1, 28,28)
indices = np.where((shaped_changes[-1, :, :] > 20) & (shaped_changes[-1, :, :] < 250))
print(indices[0])
print(indices[1])
steps = np.linspace(1, 0, cfg.sampler.num_steps)
plt.gca().invert_xaxis()
#for i in range(max_number):
plt.plot(steps, shaped_changes[:, 10, 13])
plt.plot(steps, shaped_changes[:, 7, 18])
plt.plot(steps, shaped_changes[:, 9, 12])
plt.plot(steps, shaped_changes[:, 25, 10])
plt.plot(steps, shaped_changes[:, 17, 14])
plt.xlabel('Time', fontsize=13)
plt.ylabel('State',fontsize=13)
plt.legend()
plt.grid(True)
plt.savefig('PNGs/state_changes_lbjf.png')
plt.show()
"""

In [ ]:
# multiple jumps in a single dim: MNIST
"""
#np.save('PNGs/jumps_over_1_taul.npy', changes)
changes_jump_over1 = np.load('PNGs/jumps_over_1_taul.npy')
steps = np.linspace(1, 0, len(changes_jump_over1[:-1]))

plt.gca().invert_xaxis()
plt.plot(steps, changes_jump_over1[:-1], color='black')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Avg. Proportion of Multiple Jumps', fontsize=13)
plt.title('Avg. Proportion of Multiple Jumps in one Dim. for Tau-Leaping', fontsize=14)
plt.legend()
plt.grid(True)
plt.savefig('PNGs/jumps_over_1_taul.png')
plt.show()
"""

In [ ]:
# clipped jumps: MNIST
"""
#np.save('PNGs/clipped_taul.npy', changes_clipped)
changes_clipped = np.load('PNGs/clipped_taul.npy')
steps = np.linspace(1, 0, len(changes_clipped[:-15]))
plt.gca().invert_xaxis()
plt.plot(steps, changes_clipped[:-15], color='black')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Average Proportion of Clipped Jumps', fontsize=13)
plt.title('Average Proportion of Clipped Jumps for Tau-Leaping', fontsize=14)
plt.legend()
plt.grid(True)
plt.savefig("PNGs/clipped_taul.png")
plt.show()
"""

In [ ]:
# changing dim mnist
"""
#np.save('PNGs/changing_dim_lbjf.npy', changes)
#np.save('PNGs/changing_dim_taul.npy', changes_jump)
changes_jump_taul = np.load('PNGs/changing_dim_taul.npy')
changes_jump_lbjf = np.load('PNGs/changing_dim_lbjf.npy')
steps = np.linspace(1, 0, len(changes_jump_taul[:-1]))
plt.gca().invert_xaxis()
plt.plot(steps, changes_jump_lbjf[:-1], label='Euler')
plt.plot(steps, changes[:-1], label='Tau-Leaping', color='green')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Avg. Proportion of Changing Dimensions', fontsize=13)
plt.legend()
plt.grid(True)
#plt.savefig('PNGs/changing_dim.png')
plt.show()
"""

In [ ]:
#np.save(f'lib/is_mnist/data/{cfg.model.name}_{cfg.sampler.name}{cfg.sampler.num_steps}.npy', saved_samples)

In [ ]:
#np.save('lib/is_mnist/data/mnist_hollow_100exact1000.npy', saved_samples)

In [ ]:
#np.save('SavedModels/MNIST/mnist_2704LBJF1000_200.npy', saved_samples[0:200])